참조 코드: https://developer-woo.tistory.com/60

In [1]:
import re
import time
import json
import urllib.request
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

# Naver API key 입력
client_id = '8z6s2BQ5Jo8ngUHrcj9T' 
client_secret = 'RiZI5s9Ps5'


def blog_search(keyword, end_page, crawling_page_num,client_id, client_secret):
    # 검색어 입력
    keyword = keyword
    # Selenium으로 검색 페이지 불러오기
    naver_data = []
    encText = urllib.parse.quote(keyword)

    # 검색을 끝낼 페이지 입력
    # end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
    end = end_page
    if end == "":
        end = 1
    else:
        end = int(end)
    print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행합니다")

    # 한번에 가져올 페이지 입력
    #display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
    display = crawling_page_num
    if display == "":
        display = 10
    else:
        display = int(display)
    print("\n한번에 가져올 페이지 : ", display, "페이지")

    for start in range(end):
        url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start+1}&display={display+1}" # JSON 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)
        
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        
        if rescode == 200:
            response_body = response.read()
            data = json.loads(response_body.decode('utf-8'))['items']
            
            for row in data:
                if 'blog.naver' in row['link']:
                    # HTML 태그 제거
                    title = re.sub('<[^>]*>', '', row['title'])
                    if title == '전국의 가을여행 명소':
                        pass
                    url = row['link']
                    post_date = row['postdate']
                    
                    # JSON 데이터 추가
                    naver_data.append({
                        "title": title,
                        "url": url,
                        "postdate": post_date,
                    })
            time.sleep(2)
        else:
            print("Error Code:" + rescode)
    return naver_data


/Users/baengminji/Desktop/minji/cheonann/craw/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def blog_contents(url):

    # requests를 사용하여 페이지 가져오기
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # blog 게시물 내용은 ififrame을 찾기
    iframe = soup.find('iframe', id='mainFrame')

    # iframe의 src 속성 가져오기
    iframe_url = 'https://blog.naver.com' + iframe['src']

    # iframe의 실제 페이지 가져오기
    iframe_response = requests.get(iframe_url)
    iframe_content = iframe_response.text

    # 다시 iframe의 내용 파싱
    iframe_soup = BeautifulSoup(iframe_content, 'html.parser')

    # 블로그 본문 추출
    content_divs = iframe_soup.find_all('div', class_='se-component se-text se-l-default')
    text = ""
    # 각 div에서 텍스트를 추출하여 출력
    for div in content_divs:
        paragraphs = div.find_all('p')
        for p in paragraphs:
            text += p.get_text(strip=True) + " "  # 각 문단을 공백으로 연결

    return text




In [4]:
keywords = ['도솔광장 반려인 쉼터', '천안축구센터 반려인 쉼터', '백석동 반려인 쉼터','천안시 동물보호센터','2024 천안시 dog festival','천안시 펫티켓 문화교육']

In [5]:
blog = {}
end_page = 10
crawling_page_num = 50

for keyword in keywords: 
    
    # Naver API를 사용하여 검색 결과 가져오기
    data = blog_search(keyword, end_page, crawling_page_num,client_id, client_secret)

    for i in tqdm(range(len(data))):
        url = data[i]['url']
        text = blog_contents(url)
        data[i]['text'] = text

    # 첫 번째 키워드를 blog 딕셔너리에 추가
    if keyword not in blog:
        blog[keyword] = data  # 첫 번째 키워드의 데이터를 추가

    else:
        blog[keyword].extend(data)  # 기존 키워드에 새로운 데이터를 추가


 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 40/40 [00:16<00:00,  2.38it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 235/235 [01:50<00:00,  2.13it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 465/465 [02:58<00:00,  2.60it/s]



 1 ~  10 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


 87%|████████▋ | 443/510 [11:56<01:48,  1.62s/it] 


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [6]:
data

[{'title': '천안시 동물보호센터 서산시 통영시 포항시 세종유기',
  'url': 'https://blog.naver.com/minmm86415/223521948940',
  'postdate': '20240723',
  'text': '1566-8713 \u200b \u200b  \u200b \u200b 애완동물을 키우고 싶다는 생각은 어릴 적부터 내 마음속에 자리 잡고 있었어요. \u200b 어렸을 때, TV에서 동물 다큐멘터리를 보거나 친구네 집에서 귀여운 강아지와 고양이를 볼 때마다 가슴이 두근거리기도 했어요. \u200b 특히 학교에서 키우던 작은 햄스터나 토끼를 돌보면서 느꼈던 따뜻한 감정은 나에게 깊은 인상을 남겼어요. \u200b \u200b 동물과 교감하면서 느끼는 소소한 행복과 평화로움은 다른 어떤 경험과도 비교할 수 없었어요. \u200b 하지만 바쁜 일상과 여러 가지 여건 때문에 실제로 애완동물을 키우는 것은 항상 뒷전으로 미루어졌어요. \u200b \u200b 그러던 중, 몇 년 전 친한 친구가 유기견을 입양하면서 그의 경험담을 듣게 되었어요. \u200b 친구는 입양 후에 강아지와 함께하는 시간이 얼마나 소중하고 의미 있는지 이야기해주었고, 강아지가 가족의 일원으로서 주는 기쁨과 위로를 열정적으로 설명했어요. \u200b 친구의 이야기를 들으며 나도 언젠가 애완동물과 함께 생활하고 싶다는 꿈을 키우기 시작했어요. \u200b \u200b 지난 주말, 나는 평소 애정해왔던 유기견 입양을 위해 천안시동물보호센터 를 방문했어요. \u200b \u200b 서산시동물보호센터 한국동물복지연구소(K-AWL)로부터 공식 인증을 받은 신뢰할 수 있는 브랜드로, 입양과 보호 시스템이 매우 체계적으로 운영되고 있다는 점에서 큰 관심을 끌었어요. \u200b 입양 절차와 혜택, 그리고 트레이닝 및 메디컬 센터의 뛰어난 인프라까지, 이 모든 것이 세종유기동물보호센터 를 선택하게 된 이유였어요. \u200b \u200b 천안시동물보호센터 에 도착하자마

In [3]:
for keyword in keywords: 
    
    data = blog_search(keyword,client_id, client_secret)

    for i in tqdm(range(len(data))):
        url = data[i]['url']
        text = blog_contents(url)
        data[i]['text'] = text

    blog = {
        keyword: data
    }

    blog[keyword] = data

# 전국 가을 명소의 경우 제거 해야함 
# JSON 파일로 저장
with open('blog.json', 'w', encoding='utf-8') as json_file:
    json.dump(blog, json_file, ensure_ascii=False, indent=4)
    


 1 ~  2 페이지 까지 크롤링을 진행합니다

한번에 가져올 페이지 :  50 페이지


100%|██████████| 102/102 [01:53<00:00,  1.11s/it]


In [ ]:
blog = {
    keyword: data
}

blog[keyword] = data

# # JSON 파일로 저장
# with open('blog.json', 'w', encoding='utf-8') as json_file:
#     json.dump(blog, json_file, ensure_ascii=False, indent=4)


In [ ]:
data

[{'title': '충남 천안 무료 물놀이장 무료 물놀이터 도솔광장 물놀이장',
  'url': 'https://blog.naver.com/kyo1004_/223549187270',
  'postdate': '20240815',
  'text': '안녕하세요! 오늘은 천안시에서 운영하는 애견쉼터에 대한 포스팅입니다! \u200b 얼마전 천안시 인스타그램에서 ‘백석동 반려인 쉼터’가 조성됬다는 소식을 접하고 바로 가보았어요 그러나 위치는 반려인쉼터로 검색해도 찾기쉽지않았고천안시음식물쓰레기자원화시설을 네비에 치고가면 찾기 쉬워요! 가까운 백석동인줄알았는데 알고보니 백석 공단 끝쪽이더라구요ㅠㅠ 불당가는길이라 길이좀 막히지만 가봅쉬다 도착하면 백석동 반려인 쉼터 현수막이보여요 주차장은 넓지만 주차라인은 몇개없어서 주말에 붐비지않을까 싶네요 ⚠️반려인 쉼터 에티켓 읽어봐주세요 반려인 쉼터는 중소형견/대형견 입장하는곳이 달라 안심하고 뛰뛰 가능하다는점! ⚠️반려견 놀이터에 입장 불가한 반려견도있으니 오시기전 견종 확인도 해주세요 반려인 쉼터에는 배변수거함과 놀이시설, 개수대가 있어 배변봉투만 챙겨오면 될것같아요☺️ 요렇게 그늘벤치도있어서 앉아서 반려견들을 지켜볼수도있어요ㅎㅎ 본격적으로 놀아볼까! 저는 개인적으로 코코장난감을 갖고가서 무한반복 공놀이를 했답니다😂 평일대라그런지 아무도없어서 혼자 전세내고 놀았어요ㅋㅋㅋㅋ 킁킁 냄새도맡고 기분이 요랬는데 요래됬습니당~ 친구들이 더 놀러와서 신이났어요 귀로 날라갈것만같은 코코🐶 너가좋으면 누나도좋아❤️ 간만에 준우없이 코코도 자유를 즐겼네요ㅎㅎ 다만,,, 바로옆이 음식물쓰레기처리장이라 굉장히 악취가 심하고ㅠㅠ냄새때문에 더 오래못있겠어서 집으로 돌아왔구요 잔디가 깔려있지않고 다져지지않은 돌맹이땅이라 코코젤리가 다치고말았어요😭 \u200b 위치랑 개선해야할부분이 몇가지 있는 반려인쉼터지만 반려견을 키우는 견주입장으로서는 아쉬운대로 이용해야하겠죠ㅜ? \u200b 이상 백석동 반려인쉼터 후기였습니닷!! '},
 {'title': '천

In [24]:
text

['안녕하세요! 오늘은 천안시에서 운영하는 애견쉼터에 대한 포스팅입니다!',
 '\u200b',
 '얼마전 천안시 인스타그램에서 ‘백석동 반려인 쉼터’가 조성됬다는 소식을 접하고 바로 가보았어요',
 '그러나 위치는 반려인쉼터로 검색해도 찾기쉽지않았고천안시음식물쓰레기자원화시설을 네비에 치고가면 찾기 쉬워요!',
 '가까운 백석동인줄알았는데 알고보니 백석 공단 끝쪽이더라구요ㅠㅠ 불당가는길이라 길이좀 막히지만 가봅쉬다',
 '도착하면 백석동 반려인 쉼터 현수막이보여요',
 '주차장은 넓지만 주차라인은 몇개없어서 주말에 붐비지않을까 싶네요',
 '⚠️반려인 쉼터 에티켓 읽어봐주세요',
 '반려인 쉼터는 중소형견/대형견 입장하는곳이 달라 안심하고 뛰뛰 가능하다는점!',
 '⚠️반려견 놀이터에 입장 불가한 반려견도있으니 오시기전 견종 확인도 해주세요',
 '반려인 쉼터에는 배변수거함과 놀이시설, 개수대가 있어 배변봉투만 챙겨오면 될것같아요☺️',
 '요렇게 그늘벤치도있어서 앉아서 반려견들을 지켜볼수도있어요ㅎㅎ',
 '본격적으로 놀아볼까!',
 '저는 개인적으로 코코장난감을 갖고가서 무한반복 공놀이를 했답니다😂',
 '평일대라그런지 아무도없어서 혼자 전세내고 놀았어요ㅋㅋㅋㅋ',
 '킁킁 냄새도맡고',
 '기분이 요랬는데',
 '요래됬습니당~',
 '친구들이 더 놀러와서 신이났어요',
 '귀로 날라갈것만같은 코코🐶',
 '너가좋으면 누나도좋아❤️',
 '간만에 준우없이 코코도 자유를 즐겼네요ㅎㅎ',
 '다만,,, 바로옆이 음식물쓰레기처리장이라 굉장히 악취가 심하고ㅠㅠ냄새때문에 더 오래못있겠어서 집으로 돌아왔구요',
 '잔디가 깔려있지않고 다져지지않은 돌맹이땅이라 코코젤리가 다치고말았어요😭',
 '\u200b',
 '위치랑 개선해야할부분이 몇가지 있는 반려인쉼터지만 반려견을 키우는 견주입장으로서는 아쉬운대로 이용해야하겠죠ㅜ?',
 '\u200b',
 '이상 백석동 반려인쉼터 후기였습니닷!!']

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd

# ChromeOptions 객체 생성
chrome_options = webdriver.ChromeOptions()

# User-Agent 변경
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36")

# ChromeDriver 서비스 설정
service = Service("/Users/baengminji/Desktop/minji/tobigs/visualization/chromedriver-mac-arm64/chromedriver")

# WebDriver 초기화
driver = webdriver.Chrome(service=service, options=chrome_options)

# Naver API key 입력
client_id = '8z6s2BQ5Jo8ngUHrcj9T' 
client_secret = 'RiZI5s9Ps5'

# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

# 검색어 입력
keyword = input("검색할 키워드를 입력해주세요:")
encText = urllib.parse.quote(keyword)
# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
if end == "":
    end = 1
else:
    end = int(end)
print("\n1 ~", end, "페이지까지 크롤링을 진행합니다.")

# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지:", display, "페이지")


for start in range(end):
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start+1}&display={display}"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if 'blog.naver' in row['link']:
                naver_urls.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # HTML 태그 제거
                title = re.sub('<[^>]*>', '', title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:", rescode)

# 이제 수집한 naver_urls 리스트를 순회하면서 각 블로그 페이지 내용을 가져옵니다
blog_contents = []

for url in naver_urls:
    # 블로그 페이지 열기
    driver.get(url)
    time.sleep(2)  # 페이지 로드 대기 (필요에 따라 조정)

    try:
        # 블로그 글 내용이 포함된 요소를 가져오기
        post_content = driver.find_element(By.CLASS_NAME, "se-main-container").text  # 네이버 블로그의 경우 "se-main-container" 클래스가 본문을 포함하는 경우가 많습니다
        blog_contents.append(post_content)
    except Exception as e:
        print(f"Error processing {url}: {e}")
        blog_contents.append("")

# 수집된 데이터를 확인하기 위해 출력
for idx, content in enumerate(blog_contents):
    print(f"Title: {titles[idx]}")
    print(f"Date: {postdate[idx]}")
    print(f"Content: {content[:500]}...")  # 내용의 앞부분 500자만 출력
    print("="*100)

# 작업이 끝난 후 브라우저 닫기
driver.quit()



1 ~ 2 페이지까지 크롤링을 진행합니다.

한번에 가져올 페이지: 30 페이지
Error processing https://blog.naver.com/tpwls378/223220786339: Message: no such element: Unable to locate element: {"method":"css selector","selector":".se-main-container"}
  (Session info: chrome=127.0.6533.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001011cd0c8 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x00000001011c57a4 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000100dd482c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000100e18834 cxxbridge1$string$len + 367060
4   chromedriver                        0x0000000100e5048c cxxbridge1$string$len + 595500
5   chromedriver                        0x0000000100e0d474 cxxbridge1$string$len + 321044
6   chromedriver                        0x000000

In [8]:
naver_urls

['https://blog.naver.com/tpwls378/223220786339',
 'https://blog.naver.com/raon_tori/223410903996',
 'https://blog.naver.com/nancy_jo0909/223489884759',
 'https://blog.naver.com/requiem_09/223485174400',
 'https://blog.naver.com/planade/223322566719',
 'https://blog.naver.com/hosikchoi79/223248991907',
 'https://blog.naver.com/raon_tori/223410903996',
 'https://blog.naver.com/nancy_jo0909/223489884759',
 'https://blog.naver.com/requiem_09/223485174400',
 'https://blog.naver.com/planade/223322566719',
 'https://blog.naver.com/hosikchoi79/223248991907']

In [7]:
blog_contents

['', '', '', '', '', '', '', '', '', '', '']

In [ ]:
title, url, content, tag json 파일로 크롤링 예정! 

In [4]:
titles

['천안시 백석동 반려인쉼터/애견쉼터/애견놀이터/애견운동장',
 '천안 무료 애견운동장 백석동 반려인 쉼터 후기',
 '천안시 펫티켓 문화교육 3회차 후기',
 '산책 ㅣ 천안생활체육공원 도시바람길숲, 천안축구센터 방문 후기',
 '전문 대행사는 이렇게! 천안시 K-DOG FESTIVAL 후기',
 '대한민국♡축제및동행이야기',
 '천안 무료 애견운동장 백석동 반려인 쉼터 후기',
 '천안시 펫티켓 문화교육 3회차 후기',
 '산책 ㅣ 천안생활체육공원 도시바람길숲, 천안축구센터 방문 후기',
 '전문 대행사는 이렇게! 천안시 K-DOG FESTIVAL 후기',
 '대한민국♡축제및동행이야기']

In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # 브라우저를 띄우지 않고 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Selenium을 통해 브라우저를 띄우고 네이버 블로그 페이지에 접속
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 블로그 URL
url = 'https://blog.naver.com/tpwls378/223220786339'
driver.get(url)

# 네이버 블로그는 본문이 iframe 안에 들어있기 때문에 iframe으로 전환
driver.switch_to.frame("mainFrame")

# 페이지 소스 가져오기
page_source = driver.page_source

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(page_source, 'html.parser')

# 본문 내용 추출
content_divs = soup.find_all('div', class_='se-component se-text se-l-default')

# 각 div에서 텍스트를 추출하여 출력
for div in content_divs:
    paragraphs = div.find_all('p')
    for p in paragraphs:
        text = p.get_text(strip=True)
        print(text)

# Selenium 드라이버 종료
driver.quit()


WebDriverException: Message: Can not connect to the Service /Users/baengminji/.wdm/drivers/chromedriver/mac64/127.0.6533.119/chromedriver-mac-arm64/chromedriver


In [6]:
naver_urls

['https://blog.naver.com/tpwls378/223220786339',
 'https://blog.naver.com/raon_tori/223410903996',
 'https://blog.naver.com/nancy_jo0909/223489884759',
 'https://blog.naver.com/requiem_09/223485174400',
 'https://blog.naver.com/planade/223322566719',
 'https://blog.naver.com/hosikchoi79/223248991907',
 'https://blog.naver.com/raon_tori/223410903996',
 'https://blog.naver.com/nancy_jo0909/223489884759',
 'https://blog.naver.com/requiem_09/223485174400',
 'https://blog.naver.com/planade/223322566719',
 'https://blog.naver.com/hosikchoi79/223248991907']

* 네이버 블로그
* 구글, 다음 확장 -> 데이터 수집 

어떻게 전처리? 

text html 파일로 들어올 것 같음 -> 어떤 식 정제? ()

-> 방법?? 각자 생각해오기 